In [1]:
from langchain import hub
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate

In [2]:
local_llm = "llama3.2:latest"
#url_llm = "http://172.17.30.133:11434" # ordenados con 2 gpus
#url_llm = "http://172.17.30.172:11434" #COMPUTACION
url_llm = "http://localhost:11434"

from langchain_community.llms import Ollama
llm = Ollama(model=local_llm, base_url=url_llm, temperature=0) 

C:\Users\emolt\AppData\Local\Temp\ipykernel_21336\3471671973.py:7: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=local_llm, base_url=url_llm, temperature=0)


In [3]:
import csv
from typing import Dict, List, Optional
from langchain.document_loaders.base import BaseLoader
from langchain.docstore.document import Document

class CSVLoader(BaseLoader):
    def __init__(
        self,
        file_path: str,
        source_column: Optional[str] = None,
        metadata_columns: Optional[List[str]] = None,
        csv_args: Optional[Dict] = None,
        encoding: Optional[str] = None,
    ):
        self.file_path = file_path
        self.source_column = source_column
        self.encoding = encoding
        self.csv_args = csv_args or {}
        self.metadata_columns = metadata_columns or []

    def load(self) -> List[Document]:
        docs = []
        with open(self.file_path, newline="", encoding=self.encoding) as csvfile:
            csv_reader = csv.DictReader(csvfile, **self.csv_args)
            for i, row in enumerate(csv_reader):
                metadata = {"row": i}
                for col in self.metadata_columns:
                    if col in row:
                        metadata[col] = row[col].strip()
                content = []
                for k, v in row.items():
                    if k != self.source_column and k not in self.metadata_columns:
                        content.append(f"{k.strip()}: {v.strip()}")
                doc_content = "\n".join(content)
                doc = Document(page_content=doc_content, metadata=metadata)
                docs.append(doc)

        return docs

In [4]:
metadata_columns = ["row", "grant agreement", "project acronym", "organisation PIC", "organisation name", "Small or Medium-sized Enterprise",
                   "organisation activity type", "organisation country", "organisation role", "funding for the organisation", 
                    "organisation total participation cost", "project title", "project total cost", "funding for the project",
                   "master call", "subcall", "type of proposal", "project legal basis", "project topic"]

# Instancia el CSVLoader con el archivo CSV y las columnas de metadatos
loader = CSVLoader(
    file_path = r"C:\Users\emolt\OneDrive - UMH\MASTER\TFM\BASE\cordis_data_processed.csv",
    source_column= None,  # Opcional: columna para establecer como origen
    metadata_columns=metadata_columns,
    encoding="latin1"
)

# Carga los documentos del CSV
raw_documents = loader.load()

In [5]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="row",
        description="The row in which the data is located.",
        type="int",
    ),
    AttributeInfo(
        name="grant agreement",
        description="The grant agreement number of the project, which is a identificator of the project.",
        type="int",
    ),
    AttributeInfo(
        name="project acronym",
        description="The acronym of the proyect, which is its shorted name.",
        type="string",
    ),
    AttributeInfo(
        name="organisation PIC",
        description="The Participant Identification Code, which is a identificator of the organisation.",
        type="int",
    ),
    AttributeInfo(
        name="organisation name", 
        description="The name of the orgnanisation.", 
        type="string"
    ),
    AttributeInfo(
        name="Small or Medium-sized Enterprise", 
        description="Characteristic of the organisation related to the number of employees it has. It can be: True, False or Unknown.", 
        type="string"
    ),
    AttributeInfo(
        name="organisation activity type", 
        description="The kind of activity that the organisation carries out. It can be: PRC, REC, HES, OTH, PUB or Unknown.", 
        type="string"
    ),
    AttributeInfo(
        name="organisation country", 
        description="The country in which the organisation is established. It is codified in ISO 3166-1. It can also take the value Unknown.", 
        type="string"
    ),
    AttributeInfo(
        name="organisation role", 
        description="The role that the organisation has played in the project. It can be: participan', thirdParty, coordinator, partner or internationalPartner",
        type="string"
    ),
    AttributeInfo(
        name="funding for the organisation", 
        description="The amount of money that the organisation has received from the European Union in order to conduct the project.", 
        type="string"
    ),
    AttributeInfo(
        name="organisation total participation cost", 
        description="The amount of money that the organisation has invested in carring out the project.", 
        type="string"
    ),
    AttributeInfo(
        name="project title", 
        description="The name of the project.", 
        type="string"
    ),
    AttributeInfo(
        name="project total cost", 
        description="The total amount of money that all the participants in the project have spent to carry out the project.", 
        type="string"
    ),
    AttributeInfo(
        name="funding for the project", 
        description="The total amount of money that the European Union has provided to carry out the project.", 
        type="string"
    ),
    AttributeInfo(
        name="funding for the project", 
        description="The total amount of money that the European Union has provided to carry out the project.", 
        type="string"
    ),
    AttributeInfo(
        name="master call", 
        description="The master call within which the project has been framed.", 
        type="string"
    ),
    AttributeInfo(
        name="subcall", 
        description="The subcall within which the project has been framed.", 
        type="string"
    ),
    AttributeInfo(
        name="type of proposal", 
        description="The kind of proposal that constitutes the project.", 
        type="string"
    ),
    AttributeInfo(
        name="project legal basis", 
        description="The kind of proposal that constitutes the project.", 
        type="string"
    ),
    AttributeInfo(
        name="project topic", 
        description="The topic within which the project has been framed.", 
        type="string"
    )
]
document_content_description = "Brief summary of aspects corresponding to the participation of an organisation in a determined research project funded by the European Union within the Horison 2020 programme."

In [6]:
query_1 = "What is the objective of the project with grant agreement 740934?"
query_2 = "What is the total cost of the project with the acronym HYPERGRYD?"
query_3 = "How much funding was allocated for the project titled Transforming Research through Innovative Practices for Linked interdisciplinary Exploration?"
query_4 = "Which organisation played the role of coordinator in the grant agreement 777998?"
query_5 = "What topic does the project with the acronym INTERRFACE belong to?"
query_6 = "What legal basis was the project titled European Joint Programme on Radioactive Waste Management framed within?"
query_7 = "What type of proposal was the grant agreement 814416?"
query_8 = "To which master call was the project with the acronym G9NIGHT submitted?"
query_9 = "To which sub call was the project titled Electron Nanocrystallography submitted?"
query_10 = "Provide the grant agreement of 1 project which objective is related to artificial intelligence."
query_11 = "Provide the acronym of 1 project which objective is related to robotics."
query_12 = "Provide the title of 1 project which objective is related to geolocation."
query_13 = "Provide the objective of 1 project related to digital twin."
query_14 = "Provide the objective of 3 different projects related to corrosion."
query_15 = "Provide the title of 3 different projects which objective is related to offshore structures."
query_16 = "Provide the acronym of 3 different projects which objective is related to materials engineering."
query_17 = "Provide the grant agreement of 3 different projects which objective is related to nanocomposites."
query_18 = "Provide the name of an organisation that has participated in projects which objective is related to artificial intelligence."
query_19 = "Provide the name of an organisation which activity type is PRC and that has participated in projects which objective is related to robotics."
query_20 = "Provide the PIC of an organisation that is a small or medium enterprise and has participated in projects which objective is related to geolocation."
query_21 = "Provide the name of an organisation that has played the role of coordinator in projects which objective is related to digital twin."
query_22 = "Provide the PIC of a Spanish organisation that has participated in projects which objective is related to corrosion."
query_23 = "Provide the name of an european organisation that has participated in projects which objective is related to offshore structures."
query_24 = "Provide the PIC of an european small or medium enterprise that has participated in projects which objective is related to materials engineering."
query_25 = "Provide the name of an european small or medium enterprise that has played the role of coordinator in projects which objective is related to nanocomposites."

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=800)
documents = text_splitter.split_documents(raw_documents)

In [8]:
print(f"Número de documentos originales: {len(raw_documents)}")

Número de documentos originales: 177640


In [9]:
print(f"Número de chunks generados: {len(documents)}")

Número de chunks generados: 1060810


In [12]:
 from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
model_kwargs = {"device": "cpu"} # Al no tener NVIDIA es necesario cambiarlo model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {"normalize_embeddings": True}

emb = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

In [14]:
from langchain.vectorstores import FAISS  # o Chroma, Pinecone, etc.
# Suponiendo que `documents` es una lista de Document objects con .page_content y .metadata
vectorstore_7 = FAISS.from_documents(
    documents=documents,
    embedding=emb  
)

In [15]:
from langchain.retrievers.multi_query import MultiQueryRetriever

retriever_multi_7 = MultiQueryRetriever.from_llm(
    retriever=vectorstore_7.as_retriever(),  # usa tu vectorstore existente
    llm=llm
)

In [16]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks and an expert in research projects funded by the European Union under the Horizon 2020 programme.
Use the following context from Horizon 2020 projects to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:

"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_multi_7 = (
    {"context": retriever_multi_7 | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
import time

inicio = time.time()

#retrieved_docs_1 = retriever.get_relevant_documents(query_1)
answer_1 = rag_chain_multi_7.invoke(query_1)

fin = time.time()  # Marca de tiempo al final

tiempo_ejecucion = fin - inicio  # Calcular el tiempo de ejecución

print(f"El tiempo de ejecución fue: {tiempo_ejecucion} segundos")

El tiempo de ejecución fue: 132.85808682441711 segundos


In [21]:
answer_1

"I don't know the objective of the project with grant agreement 740934."

In [22]:
import langchain

langchain.debug = False
num_queries = 25  

for i in range(1, num_queries + 1):
    query_name = f"query_{i}"
    current_query = globals().get(query_name)
    
    if current_query is not None:
        result = retriever_multi_7.invoke(current_query)
        print(f"Resultado de la consulta {query_name}: {result} \n-----\n")
    else:
        print(f"Advertencia: {query_name} no está definida")

Resultado de la consulta query_1: [Document(id='078741db-8935-46db-9004-320a58d9e4cd', metadata={'row': '53046', 'grant agreement': '842094', 'project acronym': 'WIMPACT', 'organisation PIC': '999856116', 'organisation name': 'HEINRICH-HEINE-UNIVERSITAET DUESSELDORF', 'Small or Medium-sized Enterprise': 'False', 'organisation activity type': 'HES', 'organisation country': 'DE', 'organisation role': 'coordinator', 'funding for the organisation': '174806.4', 'organisation total participation cost': '174806.4', 'project title': "Late medieval visionary Women's IMPact in early modern Castilian spiritual Tradition", 'project total cost': '174806.4', 'funding for the project': '174806.4', 'master call': 'H2020-MSCA-IF-2018', 'subcall': 'H2020-MSCA-IF-2018', 'type of proposal': 'MSCA-IF-EF-ST', 'project legal basis': 'H2020-EU.1.3. EXCELLENT SCIENCE - Marie SkÅ\x82odowska-Curie Actions', 'project topic': 'MSCA-IF-2018 Individual Fellowships'}, page_content='cc.) present in Castile in the prem

Resultado de la consulta query_2: [Document(id='078741db-8935-46db-9004-320a58d9e4cd', metadata={'row': '53046', 'grant agreement': '842094', 'project acronym': 'WIMPACT', 'organisation PIC': '999856116', 'organisation name': 'HEINRICH-HEINE-UNIVERSITAET DUESSELDORF', 'Small or Medium-sized Enterprise': 'False', 'organisation activity type': 'HES', 'organisation country': 'DE', 'organisation role': 'coordinator', 'funding for the organisation': '174806.4', 'organisation total participation cost': '174806.4', 'project title': "Late medieval visionary Women's IMPact in early modern Castilian spiritual Tradition", 'project total cost': '174806.4', 'funding for the project': '174806.4', 'master call': 'H2020-MSCA-IF-2018', 'subcall': 'H2020-MSCA-IF-2018', 'type of proposal': 'MSCA-IF-EF-ST', 'project legal basis': 'H2020-EU.1.3. EXCELLENT SCIENCE - Marie SkÅ\x82odowska-Curie Actions', 'project topic': 'MSCA-IF-2018 Individual Fellowships'}, page_content='cc.) present in Castile in the prem

Resultado de la consulta query_3: [Document(id='680e0ab9-57cc-4b9c-ab9e-9db8dfe91510', metadata={'row': '53695', 'grant agreement': '716375', 'project acronym': 'PATRIMONIVM', 'organisation PIC': '996836312', 'organisation name': 'UNIVERSITE BORDEAUX MONTAIGNE', 'Small or Medium-sized Enterprise': 'False', 'organisation activity type': 'HES', 'organisation country': 'FR', 'organisation role': 'coordinator', 'funding for the organisation': '1493478.75', 'organisation total participation cost': '1493478.75', 'project title': 'Geography and economy of the imperial properties in the Roman World (from Augustus to Diocletian).', 'project total cost': '1493478.75', 'funding for the project': '1493478.75', 'master call': 'ERC-2016-STG', 'subcall': 'ERC-2016-STG', 'type of proposal': 'ERC-STG', 'project legal basis': 'H2020-EU.1.1. EXCELLENT SCIENCE - European Research Council (ERC)', 'project topic': 'ERC-2016-STG ERC Starting Grant'}, page_content='certain way, cultural integration of periphe

Resultado de la consulta query_4: [Document(id='078741db-8935-46db-9004-320a58d9e4cd', metadata={'row': '53046', 'grant agreement': '842094', 'project acronym': 'WIMPACT', 'organisation PIC': '999856116', 'organisation name': 'HEINRICH-HEINE-UNIVERSITAET DUESSELDORF', 'Small or Medium-sized Enterprise': 'False', 'organisation activity type': 'HES', 'organisation country': 'DE', 'organisation role': 'coordinator', 'funding for the organisation': '174806.4', 'organisation total participation cost': '174806.4', 'project title': "Late medieval visionary Women's IMPact in early modern Castilian spiritual Tradition", 'project total cost': '174806.4', 'funding for the project': '174806.4', 'master call': 'H2020-MSCA-IF-2018', 'subcall': 'H2020-MSCA-IF-2018', 'type of proposal': 'MSCA-IF-EF-ST', 'project legal basis': 'H2020-EU.1.3. EXCELLENT SCIENCE - Marie SkÅ\x82odowska-Curie Actions', 'project topic': 'MSCA-IF-2018 Individual Fellowships'}, page_content='cc.) present in Castile in the prem

Resultado de la consulta query_5: [Document(id='078741db-8935-46db-9004-320a58d9e4cd', metadata={'row': '53046', 'grant agreement': '842094', 'project acronym': 'WIMPACT', 'organisation PIC': '999856116', 'organisation name': 'HEINRICH-HEINE-UNIVERSITAET DUESSELDORF', 'Small or Medium-sized Enterprise': 'False', 'organisation activity type': 'HES', 'organisation country': 'DE', 'organisation role': 'coordinator', 'funding for the organisation': '174806.4', 'organisation total participation cost': '174806.4', 'project title': "Late medieval visionary Women's IMPact in early modern Castilian spiritual Tradition", 'project total cost': '174806.4', 'funding for the project': '174806.4', 'master call': 'H2020-MSCA-IF-2018', 'subcall': 'H2020-MSCA-IF-2018', 'type of proposal': 'MSCA-IF-EF-ST', 'project legal basis': 'H2020-EU.1.3. EXCELLENT SCIENCE - Marie SkÅ\x82odowska-Curie Actions', 'project topic': 'MSCA-IF-2018 Individual Fellowships'}, page_content='cc.) present in Castile in the prem

...

Resultado de la consulta query_21: [Document(id='078741db-8935-46db-9004-320a58d9e4cd', metadata={'row': '53046', 'grant agreement': '842094', 'project acronym': 'WIMPACT', 'organisation PIC': '999856116', 'organisation name': 'HEINRICH-HEINE-UNIVERSITAET DUESSELDORF', 'Small or Medium-sized Enterprise': 'False', 'organisation activity type': 'HES', 'organisation country': 'DE', 'organisation role': 'coordinator', 'funding for the organisation': '174806.4', 'organisation total participation cost': '174806.4', 'project title': "Late medieval visionary Women's IMPact in early modern Castilian spiritual Tradition", 'project total cost': '174806.4', 'funding for the project': '174806.4', 'master call': 'H2020-MSCA-IF-2018', 'subcall': 'H2020-MSCA-IF-2018', 'type of proposal': 'MSCA-IF-EF-ST', 'project legal basis': 'H2020-EU.1.3. EXCELLENT SCIENCE - Marie SkÅ\x82odowska-Curie Actions', 'project topic': 'MSCA-IF-2018 Individual Fellowships'}, page_content='cc.) present in Castile in the pre

Resultado de la consulta query_22: [Document(id='078741db-8935-46db-9004-320a58d9e4cd', metadata={'row': '53046', 'grant agreement': '842094', 'project acronym': 'WIMPACT', 'organisation PIC': '999856116', 'organisation name': 'HEINRICH-HEINE-UNIVERSITAET DUESSELDORF', 'Small or Medium-sized Enterprise': 'False', 'organisation activity type': 'HES', 'organisation country': 'DE', 'organisation role': 'coordinator', 'funding for the organisation': '174806.4', 'organisation total participation cost': '174806.4', 'project title': "Late medieval visionary Women's IMPact in early modern Castilian spiritual Tradition", 'project total cost': '174806.4', 'funding for the project': '174806.4', 'master call': 'H2020-MSCA-IF-2018', 'subcall': 'H2020-MSCA-IF-2018', 'type of proposal': 'MSCA-IF-EF-ST', 'project legal basis': 'H2020-EU.1.3. EXCELLENT SCIENCE - Marie SkÅ\x82odowska-Curie Actions', 'project topic': 'MSCA-IF-2018 Individual Fellowships'}, page_content='cc.) present in Castile in the pre

Resultado de la consulta query_23: [Document(id='078741db-8935-46db-9004-320a58d9e4cd', metadata={'row': '53046', 'grant agreement': '842094', 'project acronym': 'WIMPACT', 'organisation PIC': '999856116', 'organisation name': 'HEINRICH-HEINE-UNIVERSITAET DUESSELDORF', 'Small or Medium-sized Enterprise': 'False', 'organisation activity type': 'HES', 'organisation country': 'DE', 'organisation role': 'coordinator', 'funding for the organisation': '174806.4', 'organisation total participation cost': '174806.4', 'project title': "Late medieval visionary Women's IMPact in early modern Castilian spiritual Tradition", 'project total cost': '174806.4', 'funding for the project': '174806.4', 'master call': 'H2020-MSCA-IF-2018', 'subcall': 'H2020-MSCA-IF-2018', 'type of proposal': 'MSCA-IF-EF-ST', 'project legal basis': 'H2020-EU.1.3. EXCELLENT SCIENCE - Marie SkÅ\x82odowska-Curie Actions', 'project topic': 'MSCA-IF-2018 Individual Fellowships'}, page_content='cc.) present in Castile in the pre

Resultado de la consulta query_24: [Document(id='078741db-8935-46db-9004-320a58d9e4cd', metadata={'row': '53046', 'grant agreement': '842094', 'project acronym': 'WIMPACT', 'organisation PIC': '999856116', 'organisation name': 'HEINRICH-HEINE-UNIVERSITAET DUESSELDORF', 'Small or Medium-sized Enterprise': 'False', 'organisation activity type': 'HES', 'organisation country': 'DE', 'organisation role': 'coordinator', 'funding for the organisation': '174806.4', 'organisation total participation cost': '174806.4', 'project title': "Late medieval visionary Women's IMPact in early modern Castilian spiritual Tradition", 'project total cost': '174806.4', 'funding for the project': '174806.4', 'master call': 'H2020-MSCA-IF-2018', 'subcall': 'H2020-MSCA-IF-2018', 'type of proposal': 'MSCA-IF-EF-ST', 'project legal basis': 'H2020-EU.1.3. EXCELLENT SCIENCE - Marie SkÅ\x82odowska-Curie Actions', 'project topic': 'MSCA-IF-2018 Individual Fellowships'}, page_content='cc.) present in Castile in the pre

Resultado de la consulta query_25: [Document(id='f01ba368-de7f-4cbf-8a1e-00f6899fd226', metadata={'row': '54157', 'grant agreement': '892794', 'project acronym': 'READESCARTES', 'organisation PIC': '999897341', 'organisation name': "UNIVERSITA CA' FOSCARI  VENEZIA", 'Small or Medium-sized Enterprise': 'False', 'organisation activity type': 'HES', 'organisation country': 'IT', 'organisation role': 'coordinator', 'funding for the organisation': '183473.28', 'organisation total participation cost': '183473.28', 'project title': 'Reading Descartes: A Reassessment of the Shaping and Transmission of Knowledge in the Seventeenth Century', 'project total cost': '183473.28', 'funding for the project': '183473.28', 'master call': 'H2020-MSCA-IF-2019', 'subcall': 'H2020-MSCA-IF-2019', 'type of proposal': 'MSCA-IF', 'project legal basis': 'H2020-EU.1.3. EXCELLENT SCIENCE - Marie SkÅ\x82odowska-Curie Actions', 'project topic': 'MSCA-IF-2019 Individual Fellowships'}, page_content='now timely, becaus